In [2]:
import json
# import numpy as np

from underthesea import word_tokenize
# import pickle
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from unidecode import unidecode
from pyvi import ViUtils


In [3]:
df = json.loads(open("E:\Visual studio project\AI assitant\data2.json", encoding=' utf-8').read())

In [4]:
import pandas as pd
import numpy as np

def json_to_dataframe(json_data):
    data = []
    
    for intent in json_data["intents"]:
        tag = intent["tag"]
        patterns = intent["patterns"][:1]
        responses = intent["response"][:1]
        
        for pattern, response in zip(patterns, responses):
            data.append([tag, pattern, response])
    
    df = pd.DataFrame(data, columns=["tag", "pattern", "response"])
    return df

# Đọc dữ liệu từ file JSON
with open("data2.json", "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)

# Chuyển đổi thành DataFrame NumPy

df_test = json_to_dataframe(json_data)
print(df_test)

         tag                                   pattern  \
0  greetings                                  xin chào   
1    goodbye                                  tạm biệt   
2     asking                              tôi muốn hỏi   
3     search                                mở cho tôi   
4      voice  Himeko ơi, chuyển sang chế độ giọng nói.   
5       text  Himeko ơi, chuyển sang chế độ nhập lệnh.   

                                    response  
0                                      Hallu  
1                           Himeko tạm biệt!  
2  Himeko sẽ tìm kiếm câu trả lời cho bạn...  
3  Himeko sẽ tìm kiếm câu trả lời cho bạn...  
4        Himeko đã đổi sang chế độ giọng nói  
5          Himeko đã đổi sang chế độ văn bản  


In [5]:
with open("vietnamese.txt", "r", encoding="utf-8") as file:
    stop_words = [line.strip() for line in file]
print(stop_words)

['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy', 'ba', 'ba ba', 'ba bản', 'ba cùng', 'ba họ', 'ba ngày', 'ba ngôi', 'ba tăng', 'bao giờ', 'bao lâu', 'bao nhiêu', 'bao nả', 'bay biến', 'biết', 'biết bao', 'biết bao nhiêu', 'biết chắc', 'biết chừng nào', 'biết mình', 'biết mấy', 'biết thế', 'biết trước', 'biết việc', 'biết đâu', 'biết đâu chừng', 'biết đâu đấy', 'biết được', 'buổi', 'buổi làm', 'buổi mới', 'buổi ngày', 'buổi sớm', 'bà', 'bà ấy', 'bài', 'bài bác', 'bài bỏ', 'bài cái', 'bác', 'bán', 'bán cấp', 'bán dạ', 'bán thế', 'bây bẩy', 'bây chừ', 'bây giờ', 'bây nhiêu', 'bèn', 'béng', 'bên', 'bên bị', 'bên có', 'bên cạnh', 'bông', 'bước', 'bước khỏi', 'bước tới', 'bước đi', 'bạn', 'bản', 'bản bộ', 'bản riêng', 'bản thân', 'bản ý', 'bất chợt', 'bất cứ', 'bất giác', 'bất kì', 'bất kể', 'bất kỳ', 'bất luận', 'bất ngờ', 'bất nhược', 'bất quá', 'bất quá chỉ', 'bất thình lình', 'bất tử', 'bất đồ', 'bấy', 'bấy chầy', 'bấy chừ', 'bấy giờ', 'bấy lâu', 'bấy lâ

In [6]:

def checkdaucau(text):
    normalized_text = unidecode(text)
    return normalized_text != text
def themdaucau(text):
    check = checkdaucau(text)
    if check == False:
        try:
            sentence = ViUtils.add_accents(text).lower()       
        except:
            sentence = text  
    else:
        sentence = text  
    return sentence
def clean_text(text):
    return  re.sub(r'[^\w\s]', ' ', text.lower(), flags=re.UNICODE)

In [7]:
words = []
label = []
doc = []
for intent in df['intents'] :
    for pattern in intent['patterns']:
        pattern1= themdaucau(pattern)
        pattern2 = clean_text(pattern1)
        # normalized_text = TextNormalization.normalize(pattern)
        w = word_tokenize(pattern2) 
        # if x not in stop_words:
        #     words.append(x) 
        # w = nlp.clear_spec(pattern)
        
        words.append(w)   
        doc.append((w, intent['tag']))
        if intent['tag'] not in label:
            label.append(intent['tag'])


words = [word for sublist in words for word in sublist]
words = sorted(set(filter(None, words)))
label = sorted(set(filter(None, label)))

words = sorted(set([w for w in words if w != '']))
label = sorted(set([l for l in label if l != '']))
doc = [(w, i) for w, i in doc if w is not None]
len_label = len(label)
print(doc)
print(words)
print(label)

[(['xin', 'chào'], 'greetings'), (['hello'], 'greetings'), (['alo'], 'greetings'), (['himeko'], 'greetings'), (['himeko', 'ơi'], 'greetings'), (['khởi động', 'nào'], 'greetings'), (['thức', 'dậy'], 'greetings'), (['chào', 'bạn', 'ngày', 'mới', 'vui'], 'greetings'), (['xin', 'chào', 'bạn', 'đây'], 'greetings'), (['thức', 'dậy', 'nào', 'bạn'], 'greetings'), (['chào', 'bạn', 'ở', 'đây', 'đấy'], 'greetings'), (['mình', 'bắt đầu', 'rồi', 'đây'], 'greetings'), (['chào', 'bạn', 'nhé', 'hãy', 'cùng', 'làm việc'], 'greetings'), (['chào', 'thân mến'], 'greetings'), (['chào', 'buổi', 'sáng', 'tươi trẻ'], 'greetings'), (['ngày', 'mới', 'vui vẻ'], 'greetings'), (['mình', 'đây', 'rồi', 'sẵn sàng', 'đồng hành'], 'greetings'), (['cùng', 'làm việc', 'thôi'], 'greetings'), (['chào', 'đáng', 'yêu'], 'greetings'), (['thức', 'dậy', 'chưa'], 'greetings'), (['thời gian', 'làm việc', 'đây'], 'greetings'), (['chào', 'tươi trẻ'], 'greetings'), (['mình', 'đây', 'rồi', 'đấy', 'bạn', 'nhé'], 'greetings'), (['bạn',

In [8]:
def build_word_mapping(word_list,word_to_index_path='word_to_index.json'):
    word_to_index = {word: index + 2 for index, word in enumerate(word_list)}
     # Đường dẫn đến tệp JSON
    with open(word_to_index_path, 'w') as json_file:
        json.dump(word_to_index, json_file)
    return word_to_index
word_to_index = build_word_mapping(words)
print(word_to_index)



{'alo': 2, 'an lành': 3, 'biến': 4, 'biết': 5, 'buổi': 6, 'bài': 7, 'bánh': 8, 'bình tĩnh': 9, 'bình yên': 10, 'bí quyết': 11, 'bạn': 12, 'bật': 13, 'bắt đầu': 14, 'chat': 15, 'chia sẻ': 16, 'cho': 17, 'chuyển': 18, 'chào': 19, 'chăm sóc': 20, 'chưa': 21, 'chương trình': 22, 'chế độ': 23, 'chỉ': 24, 'cung cấp': 25, 'cuộc': 26, 'cuộc sống': 27, 'các': 28, 'cách': 29, 'có': 30, 'có thể': 31, 'công nghệ': 32, 'công thức': 33, 'công ty': 34, 'cùng': 35, 'cần': 36, 'cẩn thận': 37, 'cậu': 38, 'cố gắng': 39, 'của': 40, 'da': 41, 'du lịch': 42, 'dậy': 43, 'dễ thương': 44, 'dừng': 45, 'giúp': 46, 'giúp đỡ': 47, 'giọng': 48, 'giới thiệu': 49, 'giữ': 50, 'gặp': 51, 'handmade': 52, 'hay': 53, 'hello': 54, 'himeko': 55, 'hiểu': 56, 'hãy': 57, 'hướng dẫn': 58, 'hẹn': 59, 'học': 60, 'học tập': 61, 'hỏi': 62, 'hợp tác': 63, 'khoa học': 64, 'không': 65, 'khởi động': 66, 'kinh nghiệm': 67, 'kiến thức': 68, 'kết thúc': 69, 'làm': 70, 'làm giàu': 71, 'làm việc': 72, 'lên': 73, 'lại': 74, 'lập trình': 75, 

In [9]:
def text_to_ids(text, word_to_id_dict = word_to_index):
    pattern=themdaucau(text)
    # normalized_text = TextNormalization.normalize(pattern)
    words = word_tokenize(pattern)
    ids = [word_to_id_dict.get(word, -1) for word in words] 
    return [id for id in ids if id != -1]  
 
def list_to_ids(word_list,word_to_id_dict = word_to_index):
    indices = [word_to_index.get(word, 1) for word in word_list]  # Sử dụng 1 nếu từ không có trong từ điển
    return indices

# Example usage
text = "khởi động chương trình"
result = text_to_ids(text)

print(result)  # Output: [21, 7]


[66, 22]


In [10]:
label2index = {"greetings": 0, "goodbye": 1, "asking": 2, "search": 3,"voice":4,"text":5}

word2index = {}
encoded_data = []
encoded_text_doc = []
for text, label in doc:
    encoded_text = list_to_ids(text) 
    encoded_label = label2index[label]
    encoded_data.append((encoded_text, encoded_label))

print(encoded_data)

max_seq_length = max(len(text) for text, _ in encoded_data)
max_word = max(max(item) for item, _ in encoded_data)

print("Có bao nhiêu từ trong danh sách là:", max_word)
print(f"Độ dài chuỗi lớn nhất:{max_seq_length}")
print(f"tổng cặp từ:{len(encoded_data)}")

[([170, 19], 0), ([54], 0), ([2], 0), ([55], 0), ([55, 193], 0), ([66, 100], 0), ([139, 43], 0), ([19, 12, 91, 83, 160], 0), ([170, 19, 12, 182], 0), ([139, 43, 100, 12], 0), ([19, 12, 194, 182, 185], 0), ([79, 14, 112, 182], 0), ([19, 12, 95, 57, 35, 72], 0), ([19, 127], 0), ([19, 6, 118, 154], 0), ([91, 83, 161], 0), ([79, 182, 112, 119, 191], 0), ([35, 72, 129], 0), ([19, 181, 175], 0), ([139, 43, 21], 0), ([136, 72, 182], 0), ([19, 154], 0), ([79, 182, 112, 185, 12, 95], 0), ([12, 183, 119], 0), ([19, 176], 0), ([140, 100], 0), ([12, 183, 43, 21], 0), ([35, 72, 85, 129], 0), ([91, 83, 14], 0), ([12, 193, 139, 43], 0), ([19, 12, 172], 0), ([12, 183, 119, 21], 0), ([91, 83, 187, 112], 0), ([170, 19, 55, 182], 0), ([55, 182, 12, 193], 0), ([55, 139, 43, 100], 0), ([19, 12, 55, 194, 182], 0), ([55, 79, 14, 112], 0), ([55, 19, 12, 95], 0), ([19, 55, 127], 0), ([55, 193, 19, 6, 118], 0), ([19, 55, 91, 83, 160], 0), ([55, 79, 182, 112], 0), ([55, 79, 35, 72], 0), ([19, 55, 181, 175], 0), 

In [11]:
def pad_indices(indices, max_length =  max_seq_length, padding_index=0):
    padded_indices = indices + [padding_index] * (max_length - len(indices))
    return padded_indices

In [12]:

encoded_data_padding = []
for encoded_text, label in encoded_data:
    padded_text = pad_indices(encoded_text , max_seq_length)
    encoded_data_padding.append((padded_text,label))
encoded_data_padding


[([170, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([55, 193, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([66, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([139, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([19, 12, 91, 83, 160, 0, 0, 0, 0, 0, 0, 0], 0),
 ([170, 19, 12, 182, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([139, 43, 100, 12, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([19, 12, 194, 182, 185, 0, 0, 0, 0, 0, 0, 0], 0),
 ([79, 14, 112, 182, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([19, 12, 95, 57, 35, 72, 0, 0, 0, 0, 0, 0], 0),
 ([19, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([19, 6, 118, 154, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([91, 83, 161, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([79, 182, 112, 119, 191, 0, 0, 0, 0, 0, 0, 0], 0),
 ([35, 72, 129, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([19, 181, 175, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([139, 43, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0),
 ([136, 72, 182, 0, 0, 0, 0, 0, 0,

In [13]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_tensor = torch.tensor(self.data[index][0], dtype=torch.long)
        label_tensor = torch.tensor(self.data[index][1], dtype=torch.long)
        
        return input_tensor, label_tensor

In [14]:
custom_dataset = CustomDataset(encoded_data_padding)

batch_size = 341
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)


batch_input, batch_label = next(iter(data_loader))

print("Batch Input Tensor:")
print(batch_input)
print("Batch Label Tensor:")
print(batch_label)

Batch Input Tensor:
tensor([[ 55, 150,  17,  ...,   0,   0,   0],
        [ 19,  12, 194,  ...,   0,   0,   0],
        [ 55, 193,  18,  ...,   0,   0,   0],
        ...,
        [ 55, 155,  59,  ...,   0,   0,   0],
        [ 59,  51,  74,  ...,   0,   0,   0],
        [ 18, 107,  23,  ...,   0,   0,   0]])
Batch Label Tensor:
tensor([3, 0, 4, 1, 5, 5, 4, 3, 4, 4, 5, 3, 0, 0, 0, 1, 1, 5, 0, 1, 2, 5, 5, 0,
        3, 3, 3, 5, 3, 1, 4, 0, 3, 1, 4, 4, 1, 5, 4, 0, 3, 5, 1, 4, 2, 0, 2, 2,
        5, 0, 3, 2, 0, 3, 1, 0, 0, 5, 3, 2, 5, 3, 5, 4, 5, 0, 3, 0, 5, 2, 3, 3,
        5, 1, 5, 3, 1, 0, 4, 2, 2, 4, 4, 4, 3, 5, 2, 0, 4, 0, 1, 2, 1, 3, 0, 4,
        4, 5, 3, 2, 3, 2, 1, 3, 3, 1, 4, 3, 3, 0, 5, 3, 4, 0, 4, 2, 0, 3, 1, 4,
        1, 1, 3, 2, 3, 4, 5, 5, 1, 3, 1, 5, 1, 4, 0, 0, 5, 1, 5, 1, 2, 0, 3, 0,
        2, 5, 0, 2, 0, 0, 1, 4, 5, 2, 5, 3, 1, 0, 2, 2, 3, 1, 3, 5, 4, 5, 3, 2,
        0, 3, 2, 1, 3, 2, 0, 3, 3, 0, 5, 1, 3, 0, 4, 1, 3, 5, 2, 2, 5, 3, 1, 2,
        4, 2, 3, 3, 3, 5, 4, 5

In [15]:
def save_model_weights(model, filepath):
    torch.save(model.state_dict(), filepath)
    return "Đã lưu model"

In [22]:

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_classes, num_heads, num_layers):
        super(SimpleTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, num_classes)
    
    def forward(self, src):
        src = self.embedding(src)
        src = src.permute(1, 0, 2)  
        output = self.transformer_encoder(src)
        output = output.mean(dim=0)  
        logits = self.fc(output)
        return logits

vocab_size = (max_word + 2) 
d_model = 256  
num_classes = 6  
num_heads = 4
num_layers = 2
num_epochs = 25

model = SimpleTransformer(vocab_size, d_model, num_classes, num_heads, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    logits = model(batch_input)
    loss = criterion(logits, batch_label)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
# Save model weights
save_model_weights(model, 'simple_transformer_weights2.pth')



Epoch 1/25, Loss: 1.7829
Epoch 2/25, Loss: 3.0388
Epoch 3/25, Loss: 3.2167
Epoch 4/25, Loss: 2.2229
Epoch 5/25, Loss: 1.5716
Epoch 6/25, Loss: 1.0585
Epoch 7/25, Loss: 0.9319
Epoch 8/25, Loss: 0.9092
Epoch 9/25, Loss: 0.7148
Epoch 10/25, Loss: 0.5111
Epoch 11/25, Loss: 0.3775
Epoch 12/25, Loss: 0.3046
Epoch 13/25, Loss: 0.2499
Epoch 14/25, Loss: 0.1896
Epoch 15/25, Loss: 0.1416
Epoch 16/25, Loss: 0.1134
Epoch 17/25, Loss: 0.0948
Epoch 18/25, Loss: 0.0754
Epoch 19/25, Loss: 0.0569
Epoch 20/25, Loss: 0.0444
Epoch 21/25, Loss: 0.0317
Epoch 22/25, Loss: 0.0257
Epoch 23/25, Loss: 0.0195
Epoch 24/25, Loss: 0.0146
Epoch 25/25, Loss: 0.0119


'Đã lưu model'

In [23]:
# Load model weights
def load_model_weights(model, filepath):
    model.load_state_dict(torch.load(filepath))
    model.eval()
# Load model weights
loaded_model = SimpleTransformer(vocab_size, d_model, num_classes, num_heads, num_layers)
load_model_weights(loaded_model, 'simple_transformer_weights2.pth')

In [18]:
def preprocess_input(input_ids, max_sequence_length=6):
    if len(input_ids) < max_sequence_length:
        # Pad the input with zeros
        padded_input_ids = torch.cat([input_ids, torch.zeros((max_sequence_length - len(input_ids),))])
        processed_input_ids = padded_input_ids.unsqueeze(0)  # Add batch dimension
    elif len(input_ids) > max_sequence_length:
        # Trim the input to desired length
        processed_input_ids = input_ids[:max_sequence_length].unsqueeze(0)  # Add batch dimension
    else:
        processed_input_ids = input_ids.unsqueeze(0)  # Add batch dimension
    
    return processed_input_ids.int() 

In [19]:
# Inference function for single input
def predict_single_input(input_ids, model):
    with torch.no_grad():
        logits = model(input_ids)
        predicted_label = torch.argmax(logits).item()
        return predicted_label

# Example usage for single input
text = "Tôi đang tìm trang web về công ty công nghệ."
result = text_to_ids(text)
test_input_ids = torch.tensor(result) 
processed_input_ids = preprocess_input(test_input_ids)
predicted_label = predict_single_input(processed_input_ids, loaded_model)
print("label: " + str(predicted_label))


label: 3


In [20]:
{"greetings": 0, "goodbye": 1, "asking": 2, "sreach": 3}

{'greetings': 0, 'goodbye': 1, 'asking': 2, 'sreach': 3}